In [1]:
import mysql.connector
import pandas as pd
import os
import numpy as np
from datetime import date

In [58]:
connection = mysql.connector.connect(user='root',
                                     password='Geslo.01',
                                     host='host.docker.internal',
                                     database='covid',
                                     port=3307)

In [59]:
cursor = connection.cursor()

In [65]:
sql_query_1 = 'DELETE FROM country'
sql_query_2 = 'DELETE FROM cases'

In [10]:
cursor.execute(sql_query_1)
connection.commit()

In [69]:
cursor.execute(sql_query_2)
connection.commit()

In [29]:
sql_query = "INSERT INTO country (id, name) VALUES (1, 'Italy'), (2, 'USA')"
cursor.execute(sql_query)
connection.commit()
print(cursor.rowcount, 'were inserted.')

2 were inserted.


In [58]:
sql_query = 'SELECT * FROM country'
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

(1, 'Italy')
(2, 'USA')


In [75]:
italy_data = pd.read_excel('Benford_Covid_Data.xls', 
                           sheet_name=0,
                           index_col=0)

In [76]:
italy_data.head(5)

,ABR,BAS,BZ,CAL,CAM,EMR,FVG,LAZ,LIG,LOM,...,MOL,PIE,PUG,SAR,SIC,TN,TOS,UMB,VDA,VEN
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-31,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-06,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-02-21,0,0,0,0,0,0,0,0,0,15,...,0,0,0,0,0,0,0,0,0,2
2020-02-22,0,0,0,0,0,2,0,0,0,40,...,0,1,0,0,0,0,0,0,0,16
2020-02-23,0,0,0,0,0,7,0,0,0,57,...,0,2,0,0,0,0,0,0,0,7


In [77]:
italy_data.dtypes

ABR    int64
BAS    int64
BZ     int64
CAL    int64
CAM    int64
EMR    int64
FVG    int64
LAZ    int64
LIG    int64
LOM    int64
MAR    int64
MOL    int64
PIE    int64
PUG    int64
SAR    int64
SIC    int64
TN     int64
TOS    int64
UMB    int64
VDA    int64
VEN    int64
dtype: object

In [78]:
italy_data.max().loc['LOM']

3251

In [80]:
italy_data[italy_data < 0] = 0

In [81]:
(italy_data < 0).sum().sum()

0

In [85]:
for column in italy_data.columns:
    temp_column = italy_data[column]
    region = column
    for i in range(len(temp_column)):
        date = temp_column.index[i].date()
        sql_query = "INSERT INTO cases (country_id, date, region, new_cases) VALUES (%(country_id)s, %(date)s, %(region)s, %(new_cases)s)"
        entry = {'country_id': 1,
                 'date': date,
                 'region': region,
                 'new_cases': temp_column[i].item()}
        cursor.execute(sql_query, entry)
        
connection.commit()

In [87]:
sql_query = 'SELECT * FROM cases'
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result[:5]:
    print(row)

(1, datetime.date(2020, 1, 31), 'ABR', 0)
(1, datetime.date(2020, 1, 31), 'BAS', 0)
(1, datetime.date(2020, 1, 31), 'BZ', 0)
(1, datetime.date(2020, 1, 31), 'CAL', 0)
(1, datetime.date(2020, 1, 31), 'CAM', 0)


In [40]:
usa_data = pd.read_excel('Benford_Covid_Data.xls',
                         sheet_name=1,
                         index_col=0)

In [41]:
usa_data.head(5)

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
Date,,,,,,,,,,,,,,,,,,,,,
01/22/2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
01/23/2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01/24/2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01/25/2020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01/26/2020,0,0,0,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
(usa_data < 0).sum().sum()

28

In [43]:
usa_data[usa_data < 0] = 0

In [44]:
(usa_data < 0).sum().sum()

0

In [60]:
usa_index = list(usa_data.index)
for i in range(len(usa_index)):
    if isinstance(usa_index[i], str) == False:
        temp_date = usa_index[i].date()
        temp_date = temp_date.replace(day=temp_date.month, month=temp_date.day)
        usa_index[i] = temp_date
usa_index

['01/22/2020',
 '01/23/2020',
 '01/24/2020',
 '01/25/2020',
 '01/26/2020',
 '01/27/2020',
 '01/28/2020',
 '01/29/2020',
 '01/30/2020',
 '01/31/2020',
 datetime.date(2020, 2, 1),
 datetime.date(2020, 2, 2),
 datetime.date(2020, 2, 3),
 datetime.date(2020, 2, 4),
 datetime.date(2020, 2, 5),
 datetime.date(2020, 2, 6),
 datetime.date(2020, 2, 7),
 datetime.date(2020, 2, 8),
 datetime.date(2020, 2, 9),
 datetime.date(2020, 2, 10),
 datetime.date(2020, 2, 11),
 datetime.date(2020, 2, 12),
 '02/13/2020',
 '02/14/2020',
 '02/15/2020',
 '02/16/2020',
 '02/17/2020',
 '02/18/2020',
 '02/19/2020',
 '02/20/2020',
 '02/21/2020',
 '02/22/2020',
 '02/23/2020',
 '02/24/2020',
 '02/25/2020',
 '02/26/2020',
 '02/27/2020',
 '02/28/2020',
 '02/29/2020',
 datetime.date(2020, 3, 1),
 datetime.date(2020, 3, 2),
 datetime.date(2020, 3, 3),
 datetime.date(2020, 3, 4),
 datetime.date(2020, 3, 5),
 datetime.date(2020, 3, 6),
 datetime.date(2020, 3, 7),
 datetime.date(2020, 3, 8),
 datetime.date(2020, 3, 9),
 dat

In [62]:
usa_data.index = pd.to_datetime(usa_index)

In [63]:
usa_data

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-26,0,0,0,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31,112,1961,752,3212,8086,605,140,69,188,8925,...,79,3088,8839,539,913,7,0,855,220,40
2020-08-01,146,1482,662,3012,6542,458,0,79,0,9591,...,103,2225,9539,583,981,7,1738,1124,93,43
2020-08-02,144,2095,637,1445,9032,449,0,69,72,7047,...,88,1443,0,379,1324,5,631,932,119,39
2020-08-03,61,1217,787,1030,5739,252,252,39,106,4758,...,65,1009,11529,394,1145,0,543,411,119,40


In [88]:
for column in usa_data.columns:
    temp_column = usa_data[column]
    region = column
    for i in range(len(temp_column)):
        date = temp_column.index[i].date()
        sql_query = "INSERT INTO cases (country_id, date, region, new_cases) VALUES (%(country_id)s, %(date)s, %(region)s, %(new_cases)s)"
        entry = {'country_id': 2,
                 'date': date,
                 'region': region,
                 'new_cases': temp_column[i].item()}
        cursor.execute(sql_query, entry)
        
connection.commit()

## Query A

In [116]:
sql_query = "SELECT name, date, region, new_cases FROM country JOIN cases ON country.id = cases.country_id"

In [118]:
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result[-5:-1]:
    print(row)

('USA', datetime.date(2020, 8, 4), 'VT', 5)
('USA', datetime.date(2020, 8, 4), 'WA', 664)
('USA', datetime.date(2020, 8, 4), 'WI', 770)
('USA', datetime.date(2020, 8, 4), 'WV', 78)


## Query B

In [26]:
sql_query = "SELECT new_table.name, new_table.max_cases, date \
             FROM (SELECT id, name, MAX(new_cases) AS max_cases \
             FROM country JOIN cases ON country.id = cases.country_id \
             GROUP BY name, id) new_table, cases \
             WHERE cases.new_cases = new_table.max_cases AND cases.country_id = new_table.id"
             

In [27]:
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('Italy', 3251, datetime.date(2020, 3, 21))
('USA', 21665, datetime.date(2020, 4, 6))


## Query C

In [48]:
sql_query = "SELECT leading_digit, COUNT(*) \
             FROM (SELECT SUBSTRING(new_cases FROM 1 FOR 1) AS leading_digit \
             FROM cases WHERE cases.country_id = 1 AND cases.new_cases != 0) new_table \
             GROUP BY leading_digit \
             ORDER BY leading_digit ASC"

In [49]:
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('1', 326)
('2', 142)
('3', 98)
('4', 94)
('5', 91)
('6', 61)
('7', 65)
('8', 53)
('9', 50)


## Query D

In [69]:
sql_query = "SELECT leading_digit, COUNT(*) / SUM(COUNT(*)) OVER() \
             FROM (SELECT SUBSTRING(new_cases FROM 1 FOR 1) AS leading_digit \
             FROM cases WHERE cases.country_id = 2 AND cases.new_cases != 0) new_table \
             GROUP BY leading_digit \
             ORDER BY leading_digit ASC"

In [70]:
cursor.execute(sql_query)
result = cursor.fetchall()
for row in result:
    print(row)

('1', Decimal('0.2989'))
('2', Decimal('0.1700'))
('3', Decimal('0.1221'))
('4', Decimal('0.1008'))
('5', Decimal('0.0812'))
('6', Decimal('0.0685'))
('7', Decimal('0.0597'))
('8', Decimal('0.0518'))
('9', Decimal('0.0471'))


In [71]:
cursor.close()

True